In [1]:
!nvidia-smi

Mon Aug 14 19:07:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:32:00.0 Off |                  Off |
| 32%   32C    P8    26W / 450W |      0MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# %cd /root/chatglm2-6b
# !python web_demo.py

In [3]:
import pandas as pd
import numpy as np
import json
import glob
import re
import sys
import warnings
warnings.filterwarnings("ignore")

from langchain import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from torch.mps import empty_cache

import torch

from text2vec import SentenceModel
from langchain.document_loaders import DirectoryLoader, TextLoader

In [4]:
# sentences = ['如何更换花呗绑定银行卡', '花呗更改绑定银行卡']

# model = SentenceModel('shibing624/text2vec-base-chinese')
# embeddings = model.encode(sentences)
# print(embeddings)

In [5]:
files_names = glob.glob('/root/autodl-tmp/alltxt/*.txt')
files_names[:5]

['/root/autodl-tmp/alltxt/2021-03-25__深圳高速公路集团股份有限公司__600548__深高速__2020年__年度报告.txt',
 '/root/autodl-tmp/alltxt/2021-04-22__福建省闽发铝业股份有限公司__002578__闽发铝业__2020年__年度报告.txt',
 '/root/autodl-tmp/alltxt/2021-04-29__大秦铁路股份有限公司__601006__大秦铁路__2020年__年度报告.txt',
 '/root/autodl-tmp/alltxt/2020-06-06__湖北九有投资股份有限公司__600462__九有股份__2019年__年度报告.txt',
 '/root/autodl-tmp/alltxt/2022-04-22__浙江德宏汽车电子电器股份有限公司__603701__德宏股份__2021年__年度报告.txt']

In [6]:
def read_txt(index, path):
    f = open(path[index],'r')
    lines = f.readlines()
    l = []
    for i in lines:
        str_lines = str(i)[0:-1].replace(r"\n","")
        dic = eval(str_lines)
        l.append(dic)
    df = pd.DataFrame(l)
    return df
df_ = read_txt(0, files_names)
df_.head()

,page,allrow,type,inside
0,1,0,text,
1,1,1,text,深圳高速公路股份有限公司
2,1,2,text,SHENZHENEXPRESSWAYCOMPANYLIMITED
3,1,3,text,（于中华人民共和国注册成立的股份有限公司）
4,1,4,text,2020


In [30]:
read_txt(0, ['/root/autodl-tmp/alltxt/2022-04-22__华辰精密装备昆山股份有限公司__300809__华辰装备__2021年__年度报告.txt'])

,page,allrow,type,inside
0,1.0,0.0,text,
1,2.0,1.0,text,
2,4.0,3.0,text,
3,3.0,2.0,text,
4,5.0,4.0,text,
...,...,...,...,...
253,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,NaN
255,NaN,NaN,NaN,NaN
256,NaN,NaN,NaN,NaN


In [7]:
def extract_section(df):
    index = df[df.iloc[:, -1] == '目录'].index.tolist()[0]
    numbers = []
    names = []
    for i in range(index+1, index+30):
        if '.......' in df.loc[i, 'inside']:
            number = re.findall(r'\d+', df.iloc[i, -1])
            ind = df.iloc[i, -1].find('...')
            name = df.iloc[i, -1][:ind]
            numbers.append(int(number[-1]))
            names.append(name)
        elif '…………' in df.loc[i, 'inside']:
            number = re.findall(r'\d+', df.iloc[i, -1])
            ind = df.iloc[i, -1].find('…')
            name = df.iloc[i, -1][:ind]
            numbers.append(int(number[-1]))
            names.append(name)
    sections = []
    for i in range(len(numbers)):
        tmp_df = df[df.loc[:, 'page'] == numbers[i]]
        tmp_df.loc[:, 'chapter'] = names[i]
        tmp_df = tmp_df.reset_index(drop = True)
        sections.append(tmp_df)
    return sections

In [8]:
sections_0 = extract_section(df_)
sections_0[8]

,page,allrow,type,inside,chapter
0,98,2506,text,,第九节优先股相关情况
1,98,2507,页眉,2020年年度报告,第九节优先股相关情况
2,98,2508,excel,"['情况说明', '根据本公司境内及香港股份过户登记处所提供的股东名册，截至报告期末，除本节...",第九节优先股相关情况
3,98,2509,text,,第九节优先股相关情况
4,98,2510,text,六、股份限制减持情况说明,第九节优先股相关情况
5,98,2511,text,□适用√不适用,第九节优先股相关情况
6,98,2512,text,第九节优先股相关情况,第九节优先股相关情况
7,98,2513,text,□适用√不适用,第九节优先股相关情况
8,98,2514,页脚,-98-,第九节优先股相关情况


In [9]:
def find_continuous_sequences(numbers):
    sequences = np.split(numbers, np.where(np.diff(numbers) != 1)[0] + 1)
    return [seq.tolist() for seq in sequences]

def process_text(df):
    tmp = 0
    new_df = pd.DataFrame(columns=df.columns)
    for i in range(df.shape[0]):
        if df.loc[i, 'inside'][-1] == '。':
            new_row = [np.nan, np.nan, 'text', ''.join(df.loc[tmp:i, 'inside']), df.loc[0, 'chapter']]
            new_df.loc[new_df.shape[0]] = new_row
            tmp = i+1
        else:
            continue
    return new_df


def preprocessing_function(df):
    df = df.replace(to_replace=[''], value=[np.nan])
    df = df.dropna(axis=0, subset=["inside"])
    df = df.drop(df[df.type == '页眉'].index)
    df = df.drop(df[df.type == '页脚'].index)
    df = df.reset_index(drop = True)
    l = []
    for i in range(df.shape[0]):
        # if re.match(r'\d+/\d+', df.iloc[i, -1]):
        if '年年度报告' in df.loc[i, 'inside']:
            l.append(i)   # 删掉 年年度报告
        if df.loc[i, 'inside'] in '[·’!"\#$%&\'()＃！（）*+,-./:;<=>?\@，：?￥★、…．＞【】［］《》？“”‘’\[\\]^_`{|}~]+':
            l.append(i)   # 删除 单独的标点符号
    df = df.drop(l)
    df = df.reset_index(drop = True)
    
    new_df = pd.DataFrame(columns=df.columns)
    excel_index = df[df.type == 'excel'].index.tolist()
    if excel_index:
        table_index = find_continuous_sequences(excel_index)
        tmp = 0
        for t in table_index:
            tmp_ = tmp
            for i in range(tmp_, t[0]):
                if df.loc[i, 'inside'][-1] == '。':
                    new_row = [np.nan, np.nan, 'text', ''.join(df.loc[tmp:i+1, 'inside']), df.loc[0, 'chapter']]
                    new_df.loc[new_df.shape[0]] = new_row
                    tmp = i+1
                else:
                    continue
            table = '| ' + ' | '.join(eval(df.loc[t[0], 'inside']))+ ' |\n'
            table += '| ' + ' | '.join(["---"] * len(eval(df.loc[t[0], 'inside']))) + ' |\n'
            for i in t[1:]:
                table += "| " + " | ".join(eval(df.loc[i, 'inside'])) + " |\n"
            new_row = [np.nan, np.nan, 'table', table, df.loc[0, 'chapter']]
            new_df.loc[new_df.shape[0]] = new_row
            tmp = t[-1]+1
    else:
        new_df = process_text(df)
    return new_df

In [10]:
preprocessing_function(sections_0[6])

,page,allrow,type,inside,chapter
0,NaN,NaN,text,第七节重要事项一、普通股利润分配或资本公积金转增预案本公司2020年度经审计的按中国企业会计...,第七节重要事项
1,NaN,NaN,text,(一)现金分红政策的制定、执行或调整情况√适用□不适用本公司一直坚持回报股东，自上市后已连续...,第七节重要事项
2,NaN,NaN,text,根据《公司章程》，公司应本着重视股东合理回报的原则，兼顾可持续经营和发展的需要，实施积极的现...,第七节重要事项
3,NaN,NaN,text,公司制订的2020年度利润分配预案（含现金分红方案）符合《公司章程》及股东回报规划的相关要求...,第七节重要事项
4,NaN,NaN,table,| 分红年度 | 每10股送红股数（股） | 每10股派息数(元)（含税） | 每10股转增...,第七节重要事项


In [11]:
def pipeline(index, path):
    df = read_txt(index, path)
    sections = extract_section(df)
    dfs = []
    for s in sections:
        df = preprocessing_function(s)
        dfs.append(df)
    processed_df = pd.concat(dfs, ignore_index=True)
    
    company = path[index].split('_')[2]
    code = path[index].split('_')[4]
    shortname = path[index].split('_')[6]
    year = path[index].split('_')[8][:-1]
    processed_df['company'] = company
    processed_df['code'] = code
    processed_df['shortname'] = shortname
    processed_df['year'] = year
    processed_df = processed_df.drop(columns='page')
    processed_df = processed_df.drop(columns='allrow')
    return processed_df

pipeline(111, files_names)

,type,inside,chapter,company,code,shortname,year
0,text,重要提示本公司董事会、监事会及董事、监事、高级管理人员保证本报告所载资料不存在任何虚假记载、...,1公司基本情况简介,山东新华制药股份有限公司,000756,新华制药,2019
1,text,董事长张代铭先生、财务负责人侯宁先生、财务资产部经理何晓洪先生声明：保证本年度报告中财务报告...,1公司基本情况简介,山东新华制药股份有限公司,000756,新华制药,2019
2,text,所有董事均已出席了审议本报告的董事会会议。,1公司基本情况简介,山东新华制药股份有限公司,000756,新华制药,2019
3,text,"公司董事会建议以621,859,447股为基数，向全体股东每10股派发现金红利人民币1.2元...",1公司基本情况简介,山东新华制药股份有限公司,000756,新华制药,2019
4,text,本公司年度财务报告已经信永中和会计师事务所审计，并出具了标准无保留意见的审计报告。,1公司基本情况简介,山东新华制药股份有限公司,000756,新华制药,2019
...,...,...,...,...,...,...,...
56,text,9.关联交易见财务报表附注十一。10.核数师,11财务报告,山东新华制药股份有限公司,000756,新华制药,2019
57,text,10.核数师有关核数师及其酬金情况详见“公司治理报告”中“核数师酬金”一节。11.持有其他上...,11财务报告,山东新华制药股份有限公司,000756,新华制药,2019
58,table,| 证券代码 | 证券简称 | 初始投资金额 | 占该公司股权比例 | 期末账面值 | 报告...,11财务报告,山东新华制药股份有限公司,000756,新华制药,2019
59,table,"| 本年发生额 |\n| --- |\n| -707,420.67 |\n| 40,952,...",12备查文件,山东新华制药股份有限公司,000756,新华制药,2019


In [ ]:
# ve = []
# te = []
# ie = []
# for i in range(len(files_names)):
#     try:
#         pipeline(i, files_names)
#     except ValueError:
#         ve.append(i)
#         # print("ValueError:", i)
#     except TypeError:
#         te.append(i)
#         # print("TypeError:", i)
#     except IndexError:
#         ie.append(i)
#         # print("IndexError:", i)

In [23]:
def preprocessing_function_(index, path):
    df = read_txt(index, path)
    df = df.replace(to_replace=[''], value=[np.nan])
    df = df.dropna(axis=0, subset=["inside"])
    df = df.drop(df[df.type == '页眉'].index)
    df = df.drop(df[df.type == '页脚'].index)
    df = df.reset_index(drop = True)
    l = []
    for i in range(df.shape[0]):
        if '年年度报告' in df.loc[i, 'inside']:
            l.append(i)   # 删掉 年年度报告
        if df.loc[i, 'inside'] in '[·’!"\#$%&\'()＃！（）*+,-./:;<=>?\@，：?￥★、…．＞【】［］《》？“”‘’\[\\]^_`{|}~]+':
            l.append(i)   # 删除 单独的标点符号
    df = df.drop(l)
    df = df.reset_index(drop = True)
    
    company = path[index].split('_')[2]
    code = path[index].split('_')[4]
    shortname = path[index].split('_')[6]
    year = path[index].split('_')[8][:-1]
    df['company'] = company
    df['code'] = code
    df['shortname'] = shortname
    df['year'] = year
    df = df.drop(columns='page')
    df = df.drop(columns='allrow')
    
    return df

In [14]:
def save2csv(files_names, index, function):
    processed_df = function(index, files_names)
    processed_df.to_csv('/root/autodl-tmp/CSV/' + files_names[index][24:-4] + '.csv', index=False)  
    # return processed_df

In [17]:
# ve = []
# te = []
# ie = []
# for i in range(len(files_names)):
#     try:
#         save2csv(files_names, i, pipeline)
#     except ValueError:
#         ve.append(i)
#         # print("ValueError:", i)
#     except TypeError:
#         te.append(i)
#         # print("TypeError:", i)
#     except IndexError:
#         ie.append(i)
#         # print("IndexError:", i)

In [25]:
# for i in ve:
#     save2csv(files_names, i, preprocessing_function_)
    
# for i in ie:
#     save2csv(files_names, i, preprocessing_function_)

In [26]:
csvs = glob.glob('/root/autodl-tmp/CSV/*.csv')
print(len(csvs))
print(len(files_names))

11587
11587


In [24]:
test = []
with open('/root/autodl-tmp/test_questions.json', 'r') as file:
    for line in file.readlines():
        dic = json.loads(line)
        test.append(dic)
test[:5]

[{'id': 0, 'question': '能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？'},
 {'id': 1,
  'question': '请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业情况进行分析。'},
 {'id': 2, 'question': '2019年四方科技电子信箱是什么?'},
 {'id': 3, 'question': '研发费用对公司的技术创新和竞争优势有何影响？'},
 {'id': 4, 'question': '康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？'}]

In [16]:
embeddings = HuggingFaceEmbeddings(
            model_name="GanymedeNil/text2vec-large-chinese",
            model_kwargs={'device': 'cuda'})
emb = embeddings.embed_query("你好")

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


In [30]:
emb = embeddings.embed_query(pipeline(0, files_names).loc[0, 'inside'])

KeyError: 5

In [23]:
test = '\n'.join(df_.iloc[:, -1])
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)

metadatas = [{"document": 1}, {"document": 2}]

texts = text_splitter.create_documents([test])
print(texts[0])
print(texts[1])

page_content='深圳高速公路股份有限公司\nSHENZHENEXPRESSWAYCOMPANYLIMITED\n（于中华人民共和国注册成立的股份有限公司）\n2020\n年年度报告全文\n（\nA\n股）\n2021年3月24日' metadata={}
page_content='2020年年度报告\n公司代码：600548公司简称：深高速\n深圳高速公路股份有限公司\n2020年年度报告\n重要提示' metadata={}
